Copyright (c) 2023 Graphcore Ltd. All rights reserved.

Node Classification on IPU using Cluster-GCN - Training with PyTorch Geometric
==================

This notebook demonstrates training a Cluster GCN model presented in [Cluster-GCN: An Efficient Algorithm for Training Deep and Large Graph Convolutional Networks](https://arxiv.org/pdf/1905.07953.pdf) with PyTorch Geometric on the Graphcore IPU. We will use the Reddit dataset from [Inductive Representation Learning on Large Graphs](https://arxiv.org/abs/1706.02216) and train the model to predict the community a post belongs to.

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------------|----------------|
|   GNNs   |  Node Classification  | CGCN | Reddit | Training, evaluation | recommended: 4 | 20mn |

This notebook assumes some familiarity with PopTorch as well as PyTorch Geometric (PyG). For additional resources please consult:

* [PopTorch Documentation](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latest/index.html),
* [PopTorch Examples and Tutorials](https://docs.graphcore.ai/en/latest/examples.html#pytorch),
* [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/)
* [PopTorch Geometric Documentation](https://docs.graphcore.ai/projects/poptorch-geometric-user-guide/en/latest/index.html)

Requirements:

A Poplar SDK environment enabled (see the Getting Started guide for your IPU system Python packages installed with `pip install -r ../requirements.txt`

### Running on Paperspace

The Paperspace environment lets you run this notebook with no set up. To improve your experience we preload datasets and pre-install packages, this can take a few minutes, if you experience errors immediately after starting a session please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to through our community of developers using our [slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/examples).

Requirements:

* Python packages installed with `pip install -r requirements.txt`

In [ ]:
%pip install -q -r requirements.txt

Lets import the required packages:

In [ ]:
import os
import os.path as osp

import matplotlib.pyplot as plt
import numpy as np
import poptorch
import torch
import torch.nn.functional as F
from poptorch_geometric.cluster_loader import FixedSizeClusterLoader
from torch_geometric.loader import ClusterData
from torch_geometric.data.summary import Summary
from torch_geometric.datasets import Reddit
from torch_geometric.nn import SAGEConv
from tqdm import tqdm

%matplotlib inline

And for compatibility with the Paperspace environment variables we will do the following:

In [ ]:
poptorch.setLogLevel("ERR")
executable_cache_dir = (
    os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/pyg-clustergcn"
)
dataset_directory = os.getenv("DATASET_DIR", "data")

Now we are ready to start!

### Reddit Dataset

PyG provides a convenient dataset class that manages downloading the Reddit dataset to local storage. The Reddit dataset contains one single graph which contains 232,965 Reddit posts. The graph is homogeneous and undirected.

In [ ]:
reddit_root = osp.join(dataset_directory, "Reddit")
dataset = Reddit(reddit_root)

We can check the `len` on the dataset to see this is one single large graph.

In [ ]:
len(dataset)

And we can view the data within the graph. We can see there are 232965 nodes each with a feature size of 602. The dataset contains masks for training, validation and test which we will apply during those stages.

In [ ]:
dataset[0]

### Clustering

As this dataset is a single large graph the computational cost grows exponentially as the layers increase. There is also a large memory requirement to keep the entire graph and node embeddings in memory. It is therefore useful to consider a sampling approach to mitigate these problems. In this example we use cluster sampling, which attempts to group the nodes into clusters of a similar size which minimises edge cuts.

The following code clusters the original dataset into 1500 clusters using [METIS](https://epubs.siam.org/doi/10.1137/S1064827595287997).

In [ ]:
total_num_clusters = 1500

cluster_data = ClusterData(
    dataset[0], num_parts=total_num_clusters, recursive=False, save_dir=reddit_root
)

We can now see we now have multiple items in the dataset:

In [ ]:
len(cluster_data)

Each with a reduced set of the original data.

In [ ]:
cluster_data[0]

It can be useful to plot the distribution of nodes in each cluster.

In [ ]:
num_nodes_per_cluster = []
num_edges_per_cluster = []

for cluster in cluster_data:
    num_nodes_per_cluster.append(cluster.y.shape[0])
    num_edges_per_cluster.append(cluster.edge_index.shape[1])

 As you can see the number of nodes per cluster is relatively balanced.

In [ ]:
plt.hist(np.array(num_nodes_per_cluster), 20)
plt.xlabel("Number of nodes per cluster")
plt.ylabel("Counts")
plt.title("Histogram of nodes in each cluster")
plt.show()

But the number of edges per cluster is not.

In [ ]:
plt.hist(np.array(num_edges_per_cluster), 20)
plt.xlabel("Number of edges per cluster")
plt.ylabel("Counts")
plt.title("Histogram of edges in each cluster")
plt.show()

We will have to take this into consideration when loading our data for the IPU. Next we will look at how to load our clusters.

## Data Loading and Batching

A batch in the cluster GCN algorithm is created by:
* Randomly select a number of clusters
* Combine the clusters into a single graph and add the edges between the nodes in this new graph that were removed in clustering
* This is our batch, a single graph that is a selection of clusters

When using the IPU we need our inputs to be fixed size. Combining the clusters will result in a graph of a different size each batch and so we need the result of our combined clusters to be fixed size. Lets see how to do that.

Firstly, we can use `Summary` from `torch_geometric` to obtain information on each of our clusters.

In [ ]:
dataset_summary = Summary.from_dataset(cluster_data)
dataset_summary

Here you can see the maximum number of nodes and edges in each cluster. From this we can calculate the maximum number of nodes that will be in each batch. Lets set the number of clusters we will sample each batch to 3 and calculate the maximum number of nodes per batch.

In [ ]:
clusters_per_batch = 6

max_num_nodes_per_cluster = int(dataset_summary.num_nodes.max)
max_nodes_per_batch = max_num_nodes_per_cluster * clusters_per_batch + 1

max_nodes_per_batch

You can see we add an extra node to our batch, this is required so that we can add any padded edges (which are required for our batch to be fixed size) to the padded node as self loops.

We also need to calculate the maximum number of edges in a batch. As the number of edges for our combined clusters cannot be known ahead of time we approximate the number of edges required. Here we will allow a standard deviation above the mean extra for each cluster.

In [ ]:
max_edges_per_batch = int(dataset_summary.num_edges.mean) + int(
    dataset_summary.num_edges.std
)
max_edges_per_batch *= clusters_per_batch
max_edges_per_batch

Now we can use these with a fixed size dataloader from PyTorch Geometric to create a dataloader that produces batches of fixed size. Notice how we set `trim_edges` to True. This is to ensure that if our combined clusters have a number of edges greater than the number of edges we have set, then the edges will be randomly removed to achieve the requested size.

In [ ]:
train_dataloader = FixedSizeClusterLoader(
    cluster_data,
    batch_size=clusters_per_batch,
    num_nodes=max_nodes_per_batch,
    collater_args=dict(num_edges=max_edges_per_batch, trim_edges=True),
    num_workers=8,
)

Lets take a look at the first few items in the dataloader:

In [ ]:
train_dataloader_iter = iter(train_dataloader)

print(next(train_dataloader_iter))
print(next(train_dataloader_iter))

You can see that these two samples have the same sizes corresponding to our specified maximum nodes and edges per batch. Now we have our dataloader set up, we can start training our model. We will do this in the next section.

## Training

Now we are in the position to start creating and training our cluster GCN model.

### Model Architecture

We take a very simple model to demonstrate the Cluster GCN approach, this is shown below. One key thing to note is we mask out the labels by setting the target at the mask locations to `-100`, which will be ignored by default in the loss function.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_1 = SAGEConv(in_channels, 128)
        self.conv_2 = SAGEConv(128, out_channels)

    def forward(self, x, edge_index, mask=None, target=None):
        x = self.conv_1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv_2(x, edge_index)
        out = F.log_softmax(x, dim=-1)

        if self.training:
            # Mask out the nodes we don't care about
            target = torch.where(~mask, target, -100)
            return out, F.nll_loss(out, target)
        return out

Lets create the `poptorch.Options` object with device iterations set to 4. Device iterations will increase the number of loops our model runs before returning to the host and can have a positive affect on our models throughput performance. For more information refer to the following resources for additional background:
* PopTorch documentation [Efficient data batching](https://docs.graphcore.ai/projects/poptorch-user-guide/en/latestbatching.html#efficient-data-batching),
* PopTorch tutorial: [Efficient data loading](https://github.com/graphcore/tutorials/tree/sdk-release-2.5/tutorials/pytorch/tut2_efficient_data_loading),

We also enable outputting the results for each iteration as well as allowing the executable to be cached to avoid recompilation.

In [ ]:
options = poptorch.Options()
options.deviceIterations(4)
options.outputMode(poptorch.OutputMode.All)
options.enableExecutableCaching(executable_cache_dir)

We can now use those options to instantiate our dataloader again.

In [ ]:
train_dataloader = FixedSizeClusterLoader(
    cluster_data,
    batch_size=clusters_per_batch,
    num_nodes=max_nodes_per_batch,
    collater_args=dict(num_edges=max_edges_per_batch, trim_edges=True),
    num_workers=8,
    options=options,
)

Now inspecting our first two batches you can see that the items are larger than previously. This is because we have increased the device iterations to 4. PopTorch will slice this batch for us and distribute it over each of the device iterations.

In [ ]:
train_dataloader_iter = iter(train_dataloader)

print(next(train_dataloader_iter))
print(next(train_dataloader_iter))

Lets create the model and prepare for training with PopTorch.

In [ ]:
model = Net(dataset.num_features, dataset.num_classes)
model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005)
poptorch_model = poptorch.trainingModel(model, optimizer=optimizer, options=options)

Now we can run the training loop:

In [ ]:
num_epochs = 10
train_losses = torch.empty(num_epochs, len(train_dataloader))

for epoch in range(num_epochs):
    bar = tqdm(train_dataloader)
    for i, data in enumerate(bar):
        # Performs forward pass, loss function evaluation,
        # backward pass and weight update in one go on the device.
        _, mini_batch_loss = poptorch_model(
            data.x, data.edge_index, data.train_mask, data.y
        )
        train_losses[epoch, i] = float(mini_batch_loss.mean())
        bar.set_description(
            f"Epoch {epoch} training loss: {train_losses[epoch, i].item():0.6f}"
        )

Finally we can detach the training model from the IPU:

In [ ]:
poptorch_model.detachFromDevice()

And finally lets take a look at the loss curve:

In [ ]:
plt.plot(train_losses.mean(dim=1))
plt.xlabel("Epoch")
plt.ylabel("Mean loss")
plt.legend(["Training loss"])
plt.grid(True)
plt.xticks(torch.arange(0, num_epochs, 2))
plt.gcf().set_dpi(150)

We have successfully trained our simple model to do node classification on the Reddit dataset. In the next section we will see how we can run validation and test on our trained model.

## Optional - Validation and Test

Now we can run validation and test on our trained model. For this we will need to do a single execution on the full graph on the CPU. This can take a while so we have left this section commented, feel free to uncomment and run validation and test.

In [ ]:
"""
data = dataset[0]

model = Net(dataset.num_features, dataset.num_classes)
model.load_state_dict(poptorch_model.state_dict())
model.eval()
out = model.forward(data.x, data.edge_index)
y_pred = out.argmax(dim=-1)

accs = []
for mask in [data.val_mask, data.test_mask]:
    correct = y_pred[mask].eq(data.y[mask]).sum().item()
    accs.append(correct / mask.sum().item())

print("Validation accuracy: {accs[0]}")
print("Test accuracy: {accs[1]}")
"""

## Follow up

We have successfully trained a simple model to do node classification on a large graph, using sampling to reduce the size of our batch.

Next you could try:
* Experiment with the dataloading to achieve higher throughput.
* Try other sampling approaches with our PopTorch Geometric tools to achieve fixed size outputs.